In [1]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions
library(SDMTools)
library(gbm)
library(randomForest)
library(zipcode)
library(mltools)
library(data.table)
library(caret)

#devtools::install_github("brews/bayfoxr", lib = "/kaggle/working")
install.packages("tidycensus")
library(tidycensus)

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.3     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   1.0.0          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loaded gbm 2.1.5

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin



Attaching package: ‘mltools’


The following object is masked from ‘package:tidyr’:

    replace_na



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required

[1] "agriculture-loan-prediction"

In [2]:
#read in files
train=read.csv("../input/agriculture-loan-prediction/train_comp1_2020.csv",stringsAsFactors=FALSE)
test=read.csv("../input/agriculture-loan-prediction/test_comp1_2020.csv",stringsAsFactors=FALSE)

In [3]:
getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}

#Remove the nulls
#head(train$TYCornUnits[!is.numeric(train$TYCornUnits)])

train$TYCornUnits=abs(as.numeric(train$TYCornUnits))
train$TYCornUnits[is.na(train$TYCornUnits)]=median(train$TYCornUnits[!is.na(train$TYCornUnits)])
train$TYBeanUnits=as.numeric(train$TYBeanUnits)
train$TYBeanUnits[is.na(train$TYBeanUnits)]=median(train$TYBeanUnits[!is.na(train$TYBeanUnits)])
train$TYWheatUnits=as.numeric(train$TYWheatUnits)
train$TYWheatUnits[is.na(train$TYWheatUnits)]=median(train$TYWheatUnits[!is.na(train$TYWheatUnits)])
train$LYCornUnits=as.numeric(train$LYCornUnits)
train$LYCornUnits[is.na(train$LYCornUnits)]=median(train$LYCornUnits[!is.na(train$LYCornUnits)])
train$LYBeanUnits=as.numeric(train$LYBeanUnits)
train$LYBeanUnits[is.na(train$LYBeanUnits)]=median(train$LYBeanUnits[!is.na(train$LYBeanUnits)])
train$LYWheatUnits=as.numeric(train$LYWheatUnits)
train$LYWheatUnits[is.na(train$LYWheatUnits)]=median(train$LYWheatUnits[!is.na(train$LYWheatUnits)])
train$AT28=as.numeric(train$AT28)
train$AT28[is.na(train$AT28)]=median(train$AT28[!is.na(train$AT28)])
train$AT33=as.numeric(train$AT33)
train$AT33[is.na(train$AT33)]=median(train$AT33[!is.na(train$AT33)])
train$AT36=as.numeric(train$AT36)
train$AT36[is.na(train$AT36)]=median(train$AT36[!is.na(train$AT36)])
train$BC33=as.numeric(train$BC33)
train$BC33[is.na(train$BC33)]=median(train$AT36[!is.na(train$BC33)])
train$BC98=as.numeric(train$BC98)
train$BC98[is.na(train$BC98)]=median(train$BC98[!is.na(train$BC98)])
train$G068=as.numeric(train$G068)
train$G068[is.na(train$G068)]=median(train$G068[!is.na(train$G068)])
train$G091=as.numeric(train$G091)
train$G091[is.na(train$G091)]=median(train$G091[!is.na(train$G091)])
train$G093=as.numeric(train$G093)
train$G093[is.na(train$G093)]=median(train$G093[!is.na(train$G093)])
train$G094=as.numeric(train$G094)
train$G094[is.na(train$G094)]=median(train$G094[!is.na(train$G094)])
train$G096=as.numeric(train$G096)
train$G096[is.na(train$G096)]=median(train$G096[!is.na(train$G096)])
train$MT28=as.numeric(train$MT28)
train$MT28[is.na(train$MT28)]=median(train$MT28[!is.na(train$MT28)])
train$MT36=as.numeric(train$MT36)
train$MT36[is.na(train$MT36)]=median(train$MT36[!is.na(train$MT36)])
train$RE34=as.numeric(train$RE34)
train$RE34[is.na(train$RE34)]=median(train$RE34[!is.na(train$RE34)])
train$S063=as.numeric(train$S063)
train$S063[is.na(train$S063)]=median(train$S063[!is.na(train$S063)])
train$Zip_Code=as.numeric(train$Zip_Code)
train$Zip_Code[is.na(train$Zip_Code)]=getmode(train$Zip_Code[!is.na(train$Zip_Code)])

#train$TYCornUnits=log(abs(train$TYCornUnits+1))
#train$TYBeanUnits=log(abs(train$TYBeanUnits+1))
#train$TYWheatUnits=log(abs(train$TYWheatUnits+1))
#train$LYCornUnits=log(abs(train$LYCornUnits+1))
#train$LYBeanUnits=log(abs(train$LYBeanUnits+1))
#train$LYWheatUnits=log(abs(train$LYWheatUnits+1))
#train$AT28=log(abs(train$AT28+1))
#train$AT33=log(abs(train$AT33+1))
#train$AT36=log(abs(train$AT36+1))
#train$BC33=log(abs(train$BC33+1))
#train$BC98=log(abs(train$BC98+1))
#train$MT28=log(abs(train$MT28+1))
#train$MT36=log(abs(train$MT36+1))
#train$RE34=log(abs(train$RE34+1))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(e

In [4]:
test$TYCornUnits=abs(as.numeric(test$TYCornUnits))
test$TYCornUnits[is.na(test$TYCornUnits)]=median(train$TYCornUnits)
test$TYBeanUnits=as.numeric(test$TYBeanUnits)
test$TYBeanUnits[is.na(test$TYBeanUnits)]=median(train$TYBeanUnits)
test$TYWheatUnits=as.numeric(test$TYWheatUnits)
test$TYWheatUnits[is.na(test$TYWheatUnits)]=median(train$TYWheatUnits)
test$LYCornUnits=as.numeric(test$LYCornUnits)
test$LYCornUnits[is.na(test$LYCornUnits)]=median(train$LYCornUnits)
test$LYBeanUnits=as.numeric(test$LYBeanUnits)
test$LYBeanUnits[is.na(test$LYBeanUnits)]=median(train$LYBeanUnits)
test$LYWheatUnits=as.numeric(test$LYWheatUnits)
test$LYWheatUnits[is.na(test$LYWheatUnits)]=median(train$LYWheatUnits)
test$AT28=as.numeric(test$AT28)
test$AT28[is.na(test$AT28)]=median(train$AT28)
test$AT33=as.numeric(test$AT33)
test$AT33[is.na(test$AT33)]=median(train$AT33)
test$AT36=as.numeric(test$AT36)
test$AT36[is.na(test$AT36)]=median(train$AT36)
test$BC33=as.numeric(test$BC33)
test$BC33[is.na(test$BC33)]=median(train$BC33)
test$BC98=as.numeric(test$BC98)
test$BC98[is.na(test$BC98)]=median(train$BC98)
test$G068=as.numeric(test$G068)
test$G068[is.na(test$G068)]=median(train$G068)
test$G091=as.numeric(test$G091)
test$G091[is.na(test$G091)]=median(train$G091)
test$G093=as.numeric(test$G093)
test$G093[is.na(test$G093)]=median(train$G093)
test$G094=as.numeric(test$G094)
test$G094[is.na(test$G094)]=median(train$G094)
test$G096=as.numeric(test$G096)
test$G096[is.na(test$G096)]=median(train$G096)
test$MT28=as.numeric(test$MT28)
test$MT28[is.na(test$MT28)]=median(train$MT28)
test$MT36=as.numeric(test$MT36)
test$MT36[is.na(test$MT36)]=median(train$MT36)
test$RE34=as.numeric(test$RE34)
test$RE34[is.na(test$RE34)]=median(train$RE34)
test$S063=as.numeric(test$S063)
test$S063[is.na(test$S063)]=median(train$S063)


#test$TYCornUnits=log(abs(test$TYCornUnits+.01))
#test$TYBeanUnits=log(abs(test$TYBeanUnits+.01))
#test$TYWheatUnits=log(abs(test$TYWheatUnits+.01))
#test$LYCornUnits=log(abs(test$LYCornUnits+.01))
#test$LYBeanUnits=log(abs(test$LYBeanUnits+.01))
#test$LYWheatUnits=log(abs(test$LYWheatUnits+.01))
#test$AT28=log(abs(test$AT28+.01))
#test$AT33=log(abs(test$AT33+.01))
#test$AT36=log(abs(test$AT36+.01))
#test$BC33=log(abs(test$BC33+.01))
#test$BC98=log(abs(test$BC98+.01))
#test$MT28=log(abs(test$MT28+.01))
#test$MT36=log(abs(test$MT36+.01))
#test$RE34=log(abs(test$RE34+.01))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(e

In [5]:
#Remove the bad bois
train$S063 <- NULL
train$MT26 <- NULL
train$G096 <- NULL
train$G091 <- NULL
train$G093 <- NULL
train$G068 <- NULL
train$COLL <- NULL
train$BC33 <- NULL
train$BC98 <- NULL
train$AT33 <- NULL

test$S063 <- NULL
test$MT26 <- NULL
test$G096 <- NULL
test$G091 <- NULL
test$G093 <- NULL
test$G068 <- NULL
test$COLL <- NULL
test$BC33 <- NULL
test$BC98 <- NULL
test$AT33 <- NULL

In [6]:
#Census Wrangling

census_api_key("0e29c36736630228c3e7f39191458ccad51de0e1")
v17 <- load_variables(2017, "acs5", cache = TRUE)
#View(v17)

To install your API key for use in future sessions, run this function with `install = TRUE`.



if(FALSE){
    
# median household income
z <- get_acs(geography = "zcta", variables = c(houseIncome="B19013_001")
z$Zip_Code = z$GEOCODE
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$estimate[is.na(train$estimate)]=median(train$estimate[!is.na(train$estimate)])
train$houseIncome = train$estimate

#Age
z_2 <- get_acs(geography = "zcta", variables = "B01002_001E")
#Income
z_3 <- get_acs(geography = "zcta", variables = "B06011_001")
#num males
z_3 <- get_acs(geography = "zcta", variables = "B01001_002")
head(z_3)
#num females
z_3 <- get_acs(geography = "zcta", variables = "B01001_026")
head(z_3)
#median age
z_3 <- get_acs(geography = "zcta", variables = "B01002_001")
head(z_3)
#median age male
z_3 <- get_acs(geography = "zcta", variables = "B01002_002")
head(z_3)
#median age female
z_3 <- get_acs(geography = "zcta", variables = "B01002_003")
head(z_3)

             
             
#Car drove alone
z_3 <- get_acs(geography = "zcta", variables = "B08101_009")
head(z_3)
#Carpooled
z_3 <- get_acs(geography = "zcta", variables = "B08101_017")
head(z_3)
#Public Transportation
z_3 <- get_acs(geography = "zcta", variables = "B08006_008")
head(z_3)
#Num house with mortgages
z_3 <- get_acs(geography = "zcta", variables = "B25027_002")
head(z_3)
#Num house without mortgages
z_3 <- get_acs(geography = "zcta", variables = "B25027_010")
head(z_3)
#Mortgage status
z_3 <- get_acs(geography = "zcta", variables = "B25081_001")
head(z_3)
             
#male 25-34 bachelors degree
z_3 <- get_acs(geography = "zcta", variables = "B15001_017")
head(z_3)
#female 25-34 bachelors degree
z_3 <- get_acs(geography = "zcta", variables = "B15001_058")
head(z_3)
#total bachelor degrees
z_3 <- get_acs(geography = "zcta", variables = "B15011_001")
head(z_3)
#high school diploma
z_3 <- get_acs(geography = "zcta", variables = "B15003_017")
head(z_3)
#less than high school
z_3 <- get_acs(geography = "zcta", variables = "B16010_002")
head(z_3)
#total masters
z_3 <- get_acs(geography = "zcta", variables = "B15003_023")
head(z_3)
#total doctorate
z_3 <- get_acs(geography = "zcta", variables = "B15003_025")
head(z_3)
             
}

In [7]:
#HouseHold Income
z <- get_acs(geography = "zcta", variables = "B19013_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,houseIncome = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
hincome = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$houseIncome[is.na(train$houseIncome)]=median(train$houseIncome[!is.na(train$houseIncome)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$houseIncome[is.na(test$houseIncome)]=median(test$houseIncome[!is.na(test$houseIncome)])

#Age
z <- get_acs(geography = "zcta", variables = "B01002_001E")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,age = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$age[is.na(train$age)]=median(train$age[!is.na(train$age)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$age[is.na(test$age)]=median(test$age[!is.na(test$age)])

#Income
z <- get_acs(geography = "zcta", variables = "B06011_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,singIncome = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$singIncome[is.na(train$singIncome)]=median(train$singIncome[!is.na(train$singIncome)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$singIncome[is.na(test$singIncome)]=median(test$singIncome[!is.na(test$singIncome)])

#num males
z <- get_acs(geography = "zcta", variables = "B01001_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nMale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nMale[is.na(train$nMale)]=median(train$nMale[!is.na(train$nMale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nMale[is.na(test$nMale)]=median(test$nMale[!is.na(test$nMale)])

#num femaels
z <- get_acs(geography = "zcta", variables = "B01001_026")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nFemale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nFemale[is.na(train$nFemale)]=median(train$nFemale[!is.na(train$nFemale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nFemale[is.na(test$nFemale)]=median(test$nFemale[!is.na(test$nFemale)])

#median age
z <- get_acs(geography = "zcta", variables = "B01002_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,medAge = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
age = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$medAge[is.na(train$medAge)]=median(train$medAge[!is.na(train$medAge)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$medAge[is.na(test$medAge)]=median(test$medAge[!is.na(test$medAge)])

#med Age male
z <- get_acs(geography = "zcta", variables = "B01002_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,medAgeMale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$medAgeMale[is.na(train$medAgeMale)]=median(train$medAgeMale[!is.na(train$medAgeMale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$medAgeMale[is.na(test$medAgeMale)]=median(test$medAgeMale[!is.na(test$medAgeMale)])

#median age femal
z <- get_acs(geography = "zcta", variables = "B01002_003")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,medAgeFemale = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$medAgeFemale[is.na(train$medAgeFemale)]=median(train$medAgeFemale[!is.na(train$medAgeFemale)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$medAgeFemale[is.na(test$medAgeFemale)]=median(test$medAgeFemale[!is.na(test$medAgeFemale)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [8]:
#Population information - demographics
#white
z <- get_acs(geography = "zcta", variables = "B01001A_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,white = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
hincome = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$white[is.na(train$white)]=median(train$white[!is.na(train$white)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$white[is.na(test$white)]=median(test$white[!is.na(test$white)])

#black
z <- get_acs(geography = "zcta", variables = "B01001B_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,black = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$black[is.na(train$black)]=median(train$black[!is.na(train$black)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$black[is.na(test$black)]=median(test$black[!is.na(test$black)])

#native americans
z <- get_acs(geography = "zcta", variables = "B01001C_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,natAm = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$natAm[is.na(train$natAm)]=median(train$natAm[!is.na(train$natAm)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$natAm[is.na(test$natAm)]=median(test$natAm[!is.na(test$natAm)])

#asian
z <- get_acs(geography = "zcta", variables = "B01001F_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,asian = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$asian[is.na(train$asian)]=median(train$asian[!is.na(train$asian)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$asian[is.na(test$asian)]=median(test$asian[!is.na(test$asian)])

#hispanic
z <- get_acs(geography = "zcta", variables = "B01001I_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,hisp = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$hisp[is.na(train$hisp)]=median(train$hisp[!is.na(train$hisp)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$hisp[is.na(test$hisp)]=median(test$hisp[!is.na(test$hisp)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [9]:
#Car drove alone
z <- get_acs(geography = "zcta", variables = "B08101_009")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,cAlone = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$cAlone[is.na(train$cAlone)]=median(train$cAlone[!is.na(train$cAlone)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$cAlone[is.na(test$cAlone)]=median(test$cAlone[!is.na(test$cAlone)])

#Carpooled
z <- get_acs(geography = "zcta", variables = "B08101_017")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,cPool = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$cPool[is.na(train$cPool)]=median(train$cPool[!is.na(train$cPool)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$cPool[is.na(test$cPool)]=median(test$cPool[!is.na(test$cPool)])

#public transportation
z <- get_acs(geography = "zcta", variables = "B08006_008")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,cPub = estimate)
pub = z
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$cPub[is.na(train$cPub)]=median(train$cPub[!is.na(train$cPub)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$cPub[is.na(test$cPub)]=median(test$cPub[!is.na(test$cPub)])


#num Mortgages
z <- get_acs(geography = "zcta", variables = "B25027_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nMort = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
mort = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nMort[is.na(train$nMort)]=median(train$nMort[!is.na(train$nMort)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nMort[is.na(test$nMort)]=median(test$nMort[!is.na(test$nMort)])

#num no Mortgages
z <- get_acs(geography = "zcta", variables = "B25027_010")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nNoMort = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nNoMort[is.na(train$nNoMort)]=median(train$nNoMort[!is.na(train$nNoMort)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nNoMort[is.na(test$nNoMort)]=median(test$nNoMort[!is.na(test$nNoMort)])

#Mortgage Status
z <- get_acs(geography = "zcta", variables = "B25081_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,mortStat = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$mortStat[is.na(train$mortStat)]=median(train$mortStat[!is.na(train$mortStat)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$mortStat[is.na(test$mortStat)]=median(test$mortStat[!is.na(test$mortStat)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [10]:
#male 25-34 bachelors degree
z <- get_acs(geography = "zcta", variables = "B15001_017")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,mBach = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$mBach[is.na(train$mBach)]=median(train$mBach[!is.na(train$mBach)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$mBach[is.na(test$mBach)]=median(test$mBach[!is.na(test$mBach)])

#female 25-34 bachelors degree
z <- get_acs(geography = "zcta", variables = "B15001_058")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,fBach = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$fBach[is.na(train$fBach)]=median(train$fBach[!is.na(train$fBach)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$fBach[is.na(test$fBach)]=median(test$fBach[!is.na(test$fBach)])

#total Bachelors degrees
z <- get_acs(geography = "zcta", variables = "B15011_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,totBach = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$totBach[is.na(train$totBach)]=median(train$totBach[!is.na(train$totBach)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$totBach[is.na(test$totBach)]=median(test$totBach[!is.na(test$totBach)])

#high school diploma
z <- get_acs(geography = "zcta", variables = "B15003_017")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,hSchool = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
skool = z
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$hSchool[is.na(train$hSchool)]=median(train$hSchool[!is.na(train$hSchool)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$hSchool[is.na(test$hSchool)]=median(test$hSchool[!is.na(test$hSchool)])

#less than high school
z <- get_acs(geography = "zcta", variables = "B16010_002")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,noSchool = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$noSchool[is.na(train$noSchool)]=median(train$noSchool[!is.na(train$noSchool)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$noSchool[is.na(test$noSchool)]=median(test$noSchool[!is.na(test$noSchool)])

#total masters
z <- get_acs(geography = "zcta", variables = "B15003_023")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,master = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$master[is.na(train$master)]=median(train$master[!is.na(train$master)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$master[is.na(test$master)]=median(test$master[!is.na(test$master)])

#total doctorate
z <- get_acs(geography = "zcta", variables = "B15003_025")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,doc = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$doc[is.na(train$doc)]=median(train$doc[!is.na(train$doc)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$doc[is.na(test$doc)]=median(test$doc[!is.na(test$doc)])

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS

Getting data from the 2014-2018 5-year ACS



In [11]:
#total housing units
z <- get_acs(geography = "zcta", variables = "B00002_001")
z = rename(z,Zip_Code = GEOID)
z$Zip_Code = as.numeric(z$Zip_Code)
z = rename(z,nHouse = estimate)
z$NAME <- NULL
z$variable <- NULL
z$moe <- NULL
head(z)
train = merge(x=train,y=z,by = "Zip_Code",all.x=TRUE)
train$nHouse[is.na(train$nHouse)]=median(train$nHouse[!is.na(train$nHouse)])
test = merge(x=test,y=z,by = "Zip_Code",all.x=TRUE)
test$nHouse[is.na(test$nHouse)]=median(test$nHouse[!is.na(test$nHouse)])


Getting data from the 2014-2018 5-year ACS



Zip_Code,nHouse
<dbl>,<dbl>
601,600
602,1100
603,1700
606,250
610,800
612,1900


In [12]:
summary(train)

    Zip_Code       CreditScore     TYCornUnits    TYBeanUnits     
 Min.   :  2139   Min.   :  0.0   Min.   :   0   Min.   :    0.0  
 1st Qu.: 45846   1st Qu.:733.0   1st Qu.:  97   1st Qu.:  330.0  
 Median : 47987   Median :785.0   Median : 160   Median :  495.0  
 Mean   : 51649   Mean   :766.9   Mean   : 231   Mean   :  688.6  
 3rd Qu.: 61272   3rd Qu.:817.0   3rd Qu.: 248   3rd Qu.:  720.0  
 Max.   :475211   Max.   :850.0   Max.   :5940   Max.   :16335.0  
  TYWheatUnits     LYCornUnits      LYBeanUnits       LYWheatUnits   
 Min.   :   0.0   Min.   :   0.0   Min.   :    0.0   Min.   :   0.0  
 1st Qu.: 135.0   1st Qu.:  54.0   1st Qu.:  237.0   1st Qu.: 145.0  
 Median : 135.0   Median : 108.0   Median :  450.0   Median : 145.0  
 Mean   : 151.1   Mean   : 181.5   Mean   :  718.4   Mean   : 165.1  
 3rd Qu.: 135.0   3rd Qu.: 202.0   3rd Qu.:  810.0   3rd Qu.: 145.0  
 Max.   :4850.0   Max.   :4616.0   Max.   :18303.0   Max.   :4230.0  
    State                AT28            

In [13]:
data(zipcode)
#zipcode

zipcode$Zip_Code = as.numeric(zipcode$zip)

dim(train)
train = merge(x=train,y=zipcode,by = "Zip_Code",all.x=TRUE)
test = merge(x=test,y=zipcode,by = "Zip_Code",all.x=TRUE)
#Make sure I did that rite
dim(train)

#We get some null lats + longs - fix it

train$latitude[is.na(train$latitude)]=median(train$latitude[!is.na(train$latitude)])
train$longitude[is.na(train$longitude)]=median(train$longitude[!is.na(train$longitude)])
test$latitude[is.na(test$latitude)]=median(test$latitude[!is.na(test$latitude)])
test$longitude[is.na(test$longitude)]=median(test$longitude[!is.na(test$longitude)])


[1] 99517    44

[1] 99517    49

In [14]:
#Normalize the population info
train = transform(train, pop = nMale + nFemale)
test = transform(test, pop = nMale + nFemale)
#train$pop[is.zero(train$pop)]=median(train$pop[!is.zero(train$pop)])
#test$pop[is.zero(test$pop)]=median(test$pop[!is.zero(test$pop)])

train = transform(train, pnMale = nMale/pop)
test = transform(test, pnMale = nMale/pop)
train$pnMale[is.na(train$pnMale)]=median(train$pnMale[!is.na(train$pnMale)])
test$pnMale[is.na(test$pnMale)]=median(test$pnMale[!is.na(test$pnMale)])
train = transform(train, pnFemale = nFemale/pop)
test = transform(test, pnFemale = nFemale/pop)
train$pnFemale[is.na(train$pnFemale)]=median(train$pnFemale[!is.na(train$pnFemale)])
test$pnFemale[is.na(test$pnFemale)]=median(test$pnFemale[!is.na(test$pnFemale)])
#Demographics
train = transform(train, pwhite = white/pop)
test = transform(test, pwhite = white/pop)
train$pwhite[is.na(train$pwhite)]=median(train$pwhite[!is.na(train$pwhite)])
test$pwhite[is.na(test$pwhite)]=median(test$pwhite[!is.na(test$pwhite)])
train = transform(train, pblack = black/pop)
test = transform(test, pblack = black/pop)
train$pblack[is.na(train$pblack)]=median(train$pblack[!is.na(train$pblack)])
test$pblack[is.na(test$pblack)]=median(test$pblack[!is.na(test$pblack)])
train = transform(train, pnatAm = natAm/pop)
test = transform(test, pnatAm = natAm/pop)
train$pnatAm[is.na(train$pnatAm)]=median(train$pnatAm[!is.na(train$pnatAm)])
test$pnatAm[is.na(test$pnatAm)]=median(test$pnatAm[!is.na(test$pnatAm)])
train = transform(train, pasian = asian/pop)
test = transform(test, pasian = asian/pop)
train$pasian[is.na(train$pasian)]=median(train$pasian[!is.na(train$pasian)])
test$pasian[is.na(test$pasian)]=median(test$pasian[!is.na(test$pasian)])
train = transform(train, phisp = hisp/pop)
test = transform(test, phisp = hisp/pop)
train$phisp[is.na(train$phisp)]=median(train$phisp[!is.na(train$phisp)])
test$phisp[is.na(test$phisp)]=median(test$phisp[!is.na(test$phisp)])
#Transport
train = transform(train, pcAlone = cAlone/pop)
test = transform(test, pcAlone = cAlone/pop)
train$pcAlone[is.na(train$pcAlone)]=median(train$pcAlone[!is.na(train$pcAlone)])
test$pcAlone[is.na(test$pcAlone)]=median(test$pcAlone[!is.na(test$pcAlone)])
train = transform(train, pcPool = cPool/pop)
test = transform(test, pcPool = cPool/pop)
train$pcPool[is.na(train$pcPool)]=median(train$pcPool[!is.na(train$pcPool)])
test$pcPool[is.na(test$pcPool)]=median(test$pcPool[!is.na(test$pcPool)])
train = transform(train, pcPub = cPub/pop)
test = transform(test, pcPub = cPub/pop)
train$pcPub[is.na(train$pcPub)]=median(train$pcPub[!is.na(train$pcPub)])
test$pcPub[is.na(test$pcPub)]=median(test$pcPub[!is.na(test$pcPub)])
#Education
train = transform(train, pmBach = mBach/pop)
test = transform(test, pmBach = mBach/pop)
train$pmBach[is.na(train$pmBach)]=median(train$pmBach[!is.na(train$pmBach)])
test$pmBach[is.na(test$pmBach)]=median(test$pmBach[!is.na(test$pmBach)])
train = transform(train, pfBach = fBach/pop)
test = transform(test, pfBach = fBach/pop)
train$pfBach[is.na(train$pfBach)]=median(train$pfBach[!is.na(train$pfBach)])
test$pfBach[is.na(test$pfBach)]=median(test$pfBach[!is.na(test$pfBach)])
train = transform(train, ptotBach = totBach/pop)
test = transform(test, ptotBach = totBach/pop)
train$ptotBach[is.na(train$ptotBach)]=median(train$ptotBach[!is.na(train$ptotBach)])
test$ptotBach[is.na(test$ptotBach)]=median(test$ptotBach[!is.na(test$ptotBach)])
train = transform(train, phSchool = hSchool/pop)
test = transform(test, phSchool = hSchool/pop)
train$phSchool[is.na(train$phSchool)]=median(train$phSchool[!is.na(train$phSchool)])
test$phSchool[is.na(test$phSchool)]=median(test$phSchool[!is.na(test$phSchool)])
train = transform(train, pnoSchool = noSchool/pop)
test = transform(test, pnoSchool = noSchool/pop)
train$pnoSchool[is.na(train$pnoSchool)]=median(train$pnoSchool[!is.na(train$pnoSchool)])
test$pnoSchool[is.na(test$pnoSchool)]=median(test$pnoSchool[!is.na(test$pnoSchool)])
train = transform(train, pmaster = master/pop)
test = transform(test, pmaster = master/pop)
train$pmaster[is.na(train$pmaster)]=median(train$pmaster[!is.na(train$pmaster)])
test$pmaster[is.na(test$pmaster)]=median(test$pmaster[!is.na(test$pmaster)])
train = transform(train, pdoc = doc/pop)
test = transform(test, pdoc = doc/pop)
train$pdoc[is.na(train$pdoc)]=median(train$pdoc[!is.na(train$pdoc)])
test$pdoc[is.na(test$pdoc)]=median(test$pdoc[!is.na(test$pdoc)])


#Housing information
train = transform(train, pnMort = nMort/nHouse)
test = transform(test, pnMort = nMort/nHouse)
train$pnMort[is.na(train$pnMort)]=median(train$pnMort[!is.na(train$pnMort)])
test$pnMort[is.na(test$pnMort)]=median(test$pnMort[!is.na(test$pnMort)])
train = transform(train, pnNoMort = nNoMort/nHouse)
test = transform(test, pnNoMort = nNoMort/nHouse)
train$pnNoMort[is.na(train$pnNoMort)]=median(train$pnNoMort[!is.na(train$pnNoMort)])
test$pnNoMort[is.na(test$pnNoMort)]=median(test$pnNoMort[!is.na(test$pnNoMort)])
train = transform(train, pmortStat = mortStat/nHouse)
test = transform(test, pmortStat = mortStat/nHouse)
train$pmortStat[is.na(train$pmortStat)]=median(train$pmortStat[!is.na(train$pmortStat)])
test$pmortStat[is.na(test$pmortStat)]=median(test$pmortStat[!is.na(test$pmortStat)])

In [15]:
#Remove more bad bois
train$state <- NULL
train$Zip_Code <- NULL
train$zip <- NULL
train$city <- NULL

test$state <- NULL
test$Zip_Code <- NULL
test$zip <- NULL
test$city <- NULL

summary(train)

  CreditScore     TYCornUnits    TYBeanUnits       TYWheatUnits   
 Min.   :  0.0   Min.   :   0   Min.   :    0.0   Min.   :   0.0  
 1st Qu.:733.0   1st Qu.:  97   1st Qu.:  330.0   1st Qu.: 135.0  
 Median :785.0   Median : 160   Median :  495.0   Median : 135.0  
 Mean   :766.9   Mean   : 231   Mean   :  688.6   Mean   : 151.1  
 3rd Qu.:817.0   3rd Qu.: 248   3rd Qu.:  720.0   3rd Qu.: 135.0  
 Max.   :850.0   Max.   :5940   Max.   :16335.0   Max.   :4850.0  
  LYCornUnits      LYBeanUnits       LYWheatUnits       State          
 Min.   :   0.0   Min.   :    0.0   Min.   :   0.0   Length:99517      
 1st Qu.:  54.0   1st Qu.:  237.0   1st Qu.: 145.0   Class :character  
 Median : 108.0   Median :  450.0   Median : 145.0   Mode  :character  
 Mean   : 181.5   Mean   :  718.4   Mean   : 165.1                     
 3rd Qu.: 202.0   3rd Qu.:  810.0   3rd Qu.: 145.0                     
 Max.   :4616.0   Max.   :18303.0   Max.   :4230.0                     
      AT28               AT

In [16]:
head(train$id)
head(train[,15])
head(train$PastDue)
head(train[,14])

[1]  1670 61510 50649 28405 70796  3779

[1] 0 0 0 0 0 0

[1] 0 0 0 0 0 0

[1] 11.4 11.4  9.6  9.6  0.0 42.5

In [17]:
#PCA Time
pca_train = train
pca_train$id <- NULL
pca_train$PastDue <- NULL
pca <- prcomp(pca_train,center=TRUE,scale=TRUE)
train$PCA1 = pca$x[,1]
train$PCA2 = pca$x[,2]
train$PCA3 = pca$x[,3]
train$PCA4 = pca$x[,4]
train$PCA5 = pca$x[,5]
train$PCA6 = pca$x[,6]
train$PCA7 = pca$x[,7]
train$PCA8 = pca$x[,8]
train$PCA9 = pca$x[,9]
train$PCA0 = pca$x[,10]

#PCA Time
pca_train = test
pca_train$id <- NULL
pca_train$PastDue <- NULL
pca <- prcomp(pca_train,center=TRUE,scale=TRUE)
test$PCA1 = pca$x[,1]
test$PCA2 = pca$x[,2]
test$PCA3 = pca$x[,3]
test$PCA4 = pca$x[,4]
test$PCA5 = pca$x[,5]
test$PCA6 = pca$x[,6]
test$PCA7 = pca$x[,7]
test$PCA8 = pca$x[,8]
test$PCA9 = pca$x[,9]
test$PCA0 = pca$x[,10]

ERROR: Error in colMeans(x, na.rm = TRUE): 'x' must be numeric


In [18]:
new_train = train

old_train = train

sz = dim(new_train)

our_sd = .02

 

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,CreditScore = CreditScore * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,TYCornUnits = TYCornUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,TYBeanUnits = TYBeanUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,TYWheatUnits = TYWheatUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,LYCornUnits = LYCornUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,LYBeanUnits = LYBeanUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,LYWheatUnits = LYWheatUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,houseIncome = houseIncome * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,age = age * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,singIncome = singIncome * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nMale = nMale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nFemale = nFemale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,medAge = medAge * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,medAgeMale = medAgeMale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,medAgeFemale = medAgeFemale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,cAlone = cAlone * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,cPool = cPool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,cPub = cPub * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nMort = nMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,mortStat = mortStat * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nNoMort = nNoMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,mBach = mBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,fBach = fBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,totBach = totBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,hSchool = hSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,noSchool = noSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,master = master * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,doc = doc * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nHouse = nHouse * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,latitude = latitude * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,longitude = longitude * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pop = pop * normy)

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,white = white * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,black = black * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,natAm = natAm * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,asian = asian * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,hisp = hisp * normy)

 
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA1 = PCA1 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA2 = PCA2 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA3 = PCA3 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA4 = PCA4 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA5 = PCA5 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA6 = PCA6 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA7 = PCA7 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA8 = PCA8 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA9 = PCA9 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA0 = PCA0 * normy)
 

 

#Percentages

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnMale = pnMale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnFemale = pnFemale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pcAlone = pcAlone * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pcPool = pcPool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pcPub = pcPub * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnMort = pnMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pmortStat = pmortStat * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnNoMort = pnNoMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pmBach = pmBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pfBach = pfBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,ptotBach = ptotBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,phSchool = phSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnoSchool = pnoSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pmaster = pmaster * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pdoc = pdoc * normy)

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pwhite = pwhite * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pblack = pblack * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnatAm = pnatAm * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pasian = pasian * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,phisp = phisp * normy)

 

new_train$normy <- NULL

 

rbind(train,new_train)

 

summary(new_train)


 

new_train = old_train[(train$PastDue == 1),]
sz = dim(new_train)
our_sd = .02

 

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,CreditScore = CreditScore * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,TYCornUnits = TYCornUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,TYBeanUnits = TYBeanUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,TYWheatUnits = TYWheatUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,LYCornUnits = LYCornUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,LYBeanUnits = LYBeanUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,LYWheatUnits = LYWheatUnits * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,houseIncome = houseIncome * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,age = age * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,singIncome = singIncome * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nMale = nMale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nFemale = nFemale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,medAge = medAge * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,medAgeMale = medAgeMale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,medAgeFemale = medAgeFemale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,cAlone = cAlone * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,cPool = cPool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,cPub = cPub * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nMort = nMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,mortStat = mortStat * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nNoMort = nNoMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,mBach = mBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,fBach = fBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,totBach = totBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,hSchool = hSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,noSchool = noSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,master = master * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,doc = doc * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,nHouse = nHouse * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,latitude = latitude * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,longitude = longitude * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pop = pop * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)

new_train = transform(new_train,white = white * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,black = black * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,natAm = natAm * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,asian = asian * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,hisp = hisp * normy)

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA1 = PCA1 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA2 = PCA2 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA3 = PCA3 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA4 = PCA4 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA5 = PCA5 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA6 = PCA6 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA7 = PCA7 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA8 = PCA8 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA9 = PCA9 * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,PCA0 = PCA0 * normy)
 

 

 

#Percentages
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnMale = pnMale * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnFemale = pnFemale * normy)
 
 
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pcAlone = pcAlone * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pcPool = pcPool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pcPub = pcPub * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnMort = pnMort * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pmortStat = pmortStat * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnNoMort = pnNoMort * normy)
 
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pmBach = pmBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pfBach = pfBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,ptotBach = ptotBach * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,phSchool = phSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnoSchool = pnoSchool * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pmaster = pmaster * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pdoc = pdoc * normy)

new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pwhite = pwhite * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pblack = pblack * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pnatAm = pnatAm * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,pasian = pasian * normy)
new_train$normy = rnorm(sz[1],mean=1,sd=our_sd)
new_train = transform(new_train,phisp = phisp * normy)
 

new_train$normy <- NULL

 

rbind(train,new_train)

 

summary(new_train)

ERROR: Error in eval(substitute(list(...)), `_data`, parent.frame()): object 'PCA1' not found


In [19]:
#split train into train/test subsets

datum_train = sample( nrow(train) , as.integer(nrow(train)*.8) )
train_1 = train[datum_train,]
train_2 = train[-datum_train,]

summary(train)

  CreditScore     TYCornUnits    TYBeanUnits       TYWheatUnits   
 Min.   :  0.0   Min.   :   0   Min.   :    0.0   Min.   :   0.0  
 1st Qu.:733.0   1st Qu.:  97   1st Qu.:  330.0   1st Qu.: 135.0  
 Median :785.0   Median : 160   Median :  495.0   Median : 135.0  
 Mean   :766.9   Mean   : 231   Mean   :  688.6   Mean   : 151.1  
 3rd Qu.:817.0   3rd Qu.: 248   3rd Qu.:  720.0   3rd Qu.: 135.0  
 Max.   :850.0   Max.   :5940   Max.   :16335.0   Max.   :4850.0  
  LYCornUnits      LYBeanUnits       LYWheatUnits       State          
 Min.   :   0.0   Min.   :    0.0   Min.   :   0.0   Length:99517      
 1st Qu.:  54.0   1st Qu.:  237.0   1st Qu.: 145.0   Class :character  
 Median : 108.0   Median :  450.0   Median : 145.0   Mode  :character  
 Mean   : 181.5   Mean   :  718.4   Mean   : 165.1                     
 3rd Qu.: 202.0   3rd Qu.:  810.0   3rd Qu.: 145.0                     
 Max.   :4616.0   Max.   :18303.0   Max.   :4230.0                     
      AT28               AT

In [20]:
model_gbm_withPer_1 = gbm(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
                       master+doc+G094+MT36+RE34+latitude+longitude+pnMale+pnFemale+
                       pcAlone+pcPool+pcPub+pnMort+pmaster+pdoc+
                       pnNoMort+pmortStat+pmBach+pfBach+ptotBach+phSchool+pnoSchool+
                       white+black+asian+natAm+hisp+pwhite+pblack+pasian+pnatAm+phisp+
                       PCA1+PCA2+PCA3+PCA4+PCA5+PCA6+PCA7+PCA8+PCA9+PCA0,
         data=train_1,n.trees=1500,interaction.depth=7)

ERROR: Error in eval(predvars, data, env): object 'PCA1' not found


In [21]:
#model_glm = glm(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
#                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
#                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
#                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
#                       master+doc+G094+MT36+RE34+latitude+longitude+pnMale+pnFemale+
#                       pcAlone+pcPool+pcPub+pnMort+pmaster+pdoc+
#                       pnNoMort+pmortStat+pmBach+pfBach+ptotBach+phSchool+pnoSchool+
#                       white+black+asian+natAm+hisp+pwhite+pblack+pasian+pnatAm+phisp+
#                       PCA1+PCA2+PCA3+PCA4+PCA5+PCA6+PCA7+PCA8+PCA9+PCA0,
#         data=train_1,family="binomial")

In [22]:
model_gbm_withPer_2 = gbm(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
                       master+doc+G094+MT36+RE34+latitude+longitude+pnMale+pnFemale+
                       pcAlone+pcPool+pcPub+pnMort+pmaster+pdoc+
                       pnNoMort+pmortStat+pmBach+pfBach+ptotBach+phSchool+pnoSchool+
                       white+black+asian+natAm+hisp+pwhite+pblack+pasian+pnatAm+phisp,
         data=train_1,n.trees=1500,interaction.depth=8)

Distribution not specified, assuming bernoulli ...


In [23]:
model_gbm_withPer_3 = gbm(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
                       master+doc+G094+MT36+RE34+latitude+longitude+pnMale+pnFemale+
                       pcAlone+pcPool+pcPub+pnMort+pmaster+pdoc+
                       pnNoMort+pmortStat+pmBach+pfBach+ptotBach+phSchool+pnoSchool+
                       white+black+asian+natAm+hisp+pwhite+pblack+pasian+pnatAm+phisp,
         data=train_1,n.trees=1500,interaction.depth=9)

Distribution not specified, assuming bernoulli ...


In [24]:
train_2$predNo = plogis(predict(model_gbm_withPer_1,train_2,n.trees=1000))
accuracy(round(train_2$predNo),train_2$PastDue)
train_2$predYes = plogis(predict(model_gbm_withPer_2,train_2,n.trees=1000))
accuracy(round(train_2$predYes),train_2$PastDue)
train_2$predOkay = plogis(predict(model_gbm_withPer_3,train_2,n.trees=1000))
accuracy(round(train_2$predOkay),train_2$PastDue)
train_2$predNo = plogis(predict(model_gbm_withPer_1,train_2,n.trees=1500))
accuracy(round(train_2$predNo),train_2$PastDue)
train_2$predYes = plogis(predict(model_gbm_withPer_2,train_2,n.trees=1500))
accuracy(round(train_2$predYes),train_2$PastDue)
train_2$predOkay = plogis(predict(model_gbm_withPer_3,train_2,n.trees=1500))
accuracy(round(train_2$predOkay),train_2$PastDue)

ERROR: Error in predict(model_gbm_withPer_1, train_2, n.trees = 1000): object 'model_gbm_withPer_1' not found


In [25]:
#train_1$
#dddd
#model_knn = knn(PastDue~CreditScore+TYCornUnits+TYBeanUnits+TYWheatUnits+LYCornUnits+
#                       LYBeanUnits+LYWheatUnits+houseIncome+age+singIncome+nMale+nFemale+
#                       medAge+medAgeMale+medAgeFemale+cAlone+cPool+cPub+nMort+
#                       nNoMort+mortStat+mBach+fBach+totBach+hSchool+noSchool+
#                       master+doc+G094+MT36+RE34+latitude+longitude+pnMale+pnFemale+
#                       pcAlone+pcPool+pcPub+pnMort+pmaster+pdoc+
#                       pnNoMort+pmortStat+pmBach+pfBach+ptotBach+phSchool+pnoSchool+
#                       white+black+asian+natAm+hisp+pwhite+pblack+pasian+pnatAm+phisp,
#         data=train_1,family="binomial",importance=TRUE)

#pred_train_rf = predict(model_rf,train_2,type="response")

#accuracy(train_2$PastDue,pred_train_2)

In [26]:
#Binomail and quasibinomial are the only viable options
#we will use binomial and output a submission here
test$PastDue = plogis(predict(model_gbm_withPer_1,test,n.trees=1000))
write.csv(test[,c("id","PastDue")],"sub_1.csv",row.names=F)
test$PastDue <- NULL
test$PastDue = plogis(predict(model_gbm_withPer_2,test,n.trees=1000))
write.csv(test[,c("id","PastDue")],"sub_2.csv",row.names=F)
test$PastDue <- NULL
test$PastDue = plogis(predict(model_gbm_withPer_3,test,n.trees=1000))
write.csv(test[,c("id","PastDue")],"sub_3",row.names=F)
test$PastDue = predict(model_rf,test,type="response")
test$PastDue = plogis(predict(model_gbm_withPer_1,test,n.trees=1500))
write.csv(test[,c("id","PastDue")],"sub_11.csv",row.names=F)
test$PastDue <- NULL
test$PastDue = plogis(predict(model_gbm_withPer_2,test,n.trees=1500))
write.csv(test[,c("id","PastDue")],"sub_21.csv",row.names=F)
test$PastDue <- NULL
test$PastDue = plogis(predict(model_gbm_withPer_3,test,n.trees=1500))
write.csv(test[,c("id","PastDue")],"sub_31",row.names=F)
test$PastDue = predict(model_rf,test,type="response")

ERROR: Error in predict(model_gbm_withPer_1, test, n.trees = 1000): object 'model_gbm_withPer_1' not found
